In [ ]:
import pandas as pd
import ast
import random

In [ ]:
file = '/nas/ucb/henrypapadatos/Sycophancy/results/BoN_RM_Starling_scores_distribution.csv'
df = pd.read_csv(file)

df['N_response'] = df['N_response'].apply(ast.literal_eval)
df['probe_score'] = df['probe_score'].apply(ast.literal_eval)

Lambda = 0.5

df.head(3)

In [ ]:
#create a dictionary with keys 1,2,4,8,16,32; these will be the number of samples (N)
BoN = {}
for i in range(6):
    BoN[2**i] = []

for index, row in df.iterrows():
    response_list = row['N_response']
    probe_score_list = row['probe_score']
    RM_score_list = row['RM_score']

    score_list = [x - y * Lambda for x, y in zip(probe_score_list, RM_score_list)]


    #we now sample N(=key) elements from the score_list and store the best response in the BoN dictionary
    for key in BoN.keys():
        random_scores = random.sample(score_list, key)
        best_score_index = score_list.index(max(random_scores))
        #we keep inly the response, not the prompt and we remove the eos token
        best_response = response_list[best_score_index].split('GPT4 Correct Assistant: Here are my thoughts on the poem:')[1].replace('<|end_of_turn|>','')
        # print('Best response for N = ', key)     
        # print(best_response)
        # print('########################################################################')
        BoN[key].append(best_response)

#add the dictionary to the dataframe
for key in BoN.keys():
    df['BoN_'+str(key)] = BoN[key]

In [ ]:
new_file_name = file.split('.csv')[0] + f'_lambda{Lambda}.csv'
new_df = pd.DataFrame(BoN)